<a href="https://colab.research.google.com/github/paulaprado1904/TCC/blob/main/AE_MonoObjetivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ae.py**

Algoritmo mais genérico: trabalha com frases alvo.

1. Ideia geral do algoritmo

O programa funciona como uma simulação de evolução biológica:

Existe uma frase alvo (exemplo: "O Tejo e mais belo que o rio...").

Uma população inicial de frases é criada aleatoriamente, mas usando os mesmos caracteres que aparecem na frase alvo.

Cada frase tem um fitness: mede quantos caracteres estão diferentes da frase alvo. Quanto mais próximo de 0, melhor.

O algoritmo evolui assim:

Elitismo: os melhores indivíduos (frases mais parecidas com a frase alvo) passam direto para a próxima geração.

Seleção por torneio: escolhe pais para gerar filhos.

Recombinação: mistura genes dos pais (caracteres da frase).

Mutação: altera um caractere aleatório da frase, para trazer diversidade.

Isso se repete por várias gerações, até que:

O número máximo de gerações é alcançado, ou

A frase alvo é reproduzida exatamente (fitness = 0).

In [ ]:
import struct
import time
from numba import njit # Incompatível com classes, sendo necessário a utilização de funções auxiliares para algumas funções
import numpy as np

TAMANHO = 800 # Não alterar

class INDIVIDUO:
    def __init__(self, fitness = 0, fraseArray = None): #__init__ é um método construtor que inicializa os atributos dos objetos
        self.fitness = fitness # Número de caracteres distintos entre a frase alvo e a frase cópia (quanto menor melhor)
        self.fraseArray = fraseArray # Frase cópia representada como array numérico

class PARAMETROS:
    def __init__(self, fraseAlvo = "", quantidadeLetras = 0, populacao = 0, geracoes = 0, elitismo = 0, mutacao = 0, torneio = 0):
        self.fraseAlvo = fraseAlvo # Frase alvo
        self.fraseAlvoArray = None # Frase alvo representada como array numérico
        self.charToInt = {} # Mapeamento char -> int
        self.intToChar = np.array([]) # Mapeamento int -> char (array NumPy)
        self.quantidadeLetras = quantidadeLetras # Número de caracteres distintos presentes na frase alvo
        self.populacao = populacao # Tamanho da população
        self.geracoes = geracoes # Número de gerações
        self.elitismo = elitismo # Taxa de elitismo (1 a 100)
        self.mutacao = mutacao # Taxa de mutação (1 a 100)
        self.torneio = torneio # Número de participantes da seleção por torneio

'''
    Função escreveArquivo:
        Escreve no arquivo entrada.in a frase alvo e os parâmetros especificados.
'''
def escreveArquivo():
    populacao = 600 # o algoritmo vai trabalhar com 600 frases simultâneas em cada geração.
    geracoes = 1800 # máximo de 1800 ciclos de evolução.
    mutacao = 15 # taxa de mutação de 15% (chance de mudar um caractere aleatoriamente).
    elitismo = 5 # 5% da população vai direto para a próxima geração sem sofrer recombinação/mutação.
    torneio = 3 # seleção por torneio com 3 indivíduos (o melhor deles vence e é escolhido como pai/mãe).

    fraseAlvo = "O Tejo e mais belo que o rio que corre pela minha aldeia,Mas o Tejo nao e mais belo que o rio que corre pela minha aldeia.Porque o Tejo nao e o rio que corre pela minha aldeia,O Tejo tem grandes navios,E nele navega ainda,Para aqueles que veem em tudo o que la nao esta,A memoria das naus.O Tejo desce de Espanha,E o Tejo entra no mar em Portugal.Toda a gente sabe isso.Mas poucos sabem qual e o rio da minha aldeia,E para onde ele vai,E donde ele vem.E por isso, porque pertence a menos gente,e mais livre e maior o rio da minha aldeia.Pelo Tejo vai se para o Mundo.Para alem do Tejo ha a America.E a fortuna daqueles que a encontram.Ninguem nunca pensou no que ha para alem.Do rio da minha aldeia.O rio da minha aldeia nao faz pensar em nada.Quem esta ao pe dele esta so ao pe dele." #784

    # Trecho adaptado da música "De Volta Para o Futuro" de Fabio Brazza
    #fraseAlvo = "Eram la pelos anos tres mil e o mundo era mais ou menos aquilo que Nostradamus previu.O ser humano frio,num andar robotico,um olhar vazio,um mundo caotico.Eu vi a manipulacao genetica definir antes de nascer,o nosso ser e nossa estetica.Humanidade cetica,desafiando a etica,como se nao passassemos de uma mera formula aritmetica.Vi cidades sendo engolidas pelos mares com o desaparecimento das calotas polares.Eu vi cameras ate no ceu,o verdadeiro Big Brother,como descrito por George Orwel.Seguimos a natureza fria da profecia maquiavelica e a maior industria do mundo continuava sendo a belica." #596

    # Trecho do livro "Em Algum Lugar Nas Estrelas" de Clare Vanderpool
    #fraseAlvo = "A grande ursa negra,impressionante como a Ursa Maior,balancou a cabeca de um lado para o outro,e seu rugido fez tremer a paisagem proxima da Trilha Apalache.Eu digo que e ela,mas a verdade e que nao dava para ter certeza.Nao haviam marcas que inidicavam que era femea.Nao havia filhotes a vista.Mas eu sabia.Eu a conhecia como conhecia minha propria mae.Era sua vontade inabalavel de nos manter vivos." #401

    # Trecho da música "Jesus Chorou" de Racionais MC's
    #fraseAlvo = "O que e, o que e?Clara e salgada,cabe em um olho e pesa uma tonelada.Tem sabor de mar,pode ser discreta.Inquilina da dor,morada predileta.Na calada ela vem,refem da vinganca,irma do desespero,rival da esperança." #211

    try:
        # Gera o arquivo entrada.in para escrita
        with open('entrada.in', 'wb') as arquivo: # arquivo aberto em binário
            arquivo.write(populacao.to_bytes(4, byteorder='big'))
            arquivo.write(geracoes.to_bytes(4, byteorder='big'))
            arquivo.write(mutacao.to_bytes(4, byteorder='big'))
            arquivo.write(elitismo.to_bytes(4, byteorder='big'))
            arquivo.write(torneio.to_bytes(4, byteorder='big'))
            arquivo.write(fraseAlvo.encode()) #converte a string para bytes
        # Estrutura with fecha o arquivo automaticamente
    except Exception as e: #caso haja erro na criação do arquivo metricas.in
        print(f"Problemas na criação do arquivo {e}\n")

'''
    Função escreveRelatorio:
        Escreve no arquivo relatorio.txt o tempo de execução do programa e o fitness
        do melhor indivíduo obtido.
    Parâmetros:
        tempo - tempo gasto na execução do programa.
        fitness - número de caracteres distintos da frase alvo do indivíduo mais adaptado da População.
    Retorno:
        Nulo.
'''
def escreveRelatorio(tempo, fitness):
    try:
        # Gravando os dados no arquivo relatorio.txt
        with open('relatorio.txt', 'a') as arquivo, open('metricas.in', 'ab') as binario:
            arquivo.write(f'{tempo:.3f}')
            arquivo.write(f'{fitness:.3f}')
            binario.write(struct.pack("d", tempo))
            binario.write(struct.pack("f", fitness))
    except Exception as e: # Caso haja erro na criação do arquivo relatorio.txt
        print(f"Problemas na criação do arquivo {e}\n")

    try:
        with open('relatorio.txt', 'r') as arquivo:
            resultado = arquivo.read()
            if not resultado:
                print("Erro na gravação")
    except FileNotFoundError as e:
        print(f"Arquivo {e} não foi encontrado.\n")

'''
    Função leArquivo:
        Lê no arquivo entrada.in com os parâmetros definidos pelo usuário
        para os testes.
    Retorno:
        Instância da classe PARAMETROS com os parâmetros lidos.
'''
#função para ler o arquivo entrada.in
def leArquivo():
    try:
        parametros = PARAMETROS()
        # Lê o arquivo entrada.in
        with open('entrada.in', 'rb') as arquivo:
            populacao = arquivo.read(4) #deixar só arquivo.read(4)
            geracoes = arquivo.read(4)
            mutacao = arquivo.read(4)
            elitismo = arquivo.read(4)
            torneio = arquivo.read(4)
            fraseAlvo = arquivo.read(TAMANHO).decode('utf-8').rstrip('\x00')

            parametros.populacao = int.from_bytes(populacao, byteorder='big')
            parametros.geracoes = int.from_bytes(geracoes, byteorder='big')
            parametros.mutacao = int.from_bytes(mutacao, byteorder='big')
            parametros.elitismo = int.from_bytes(elitismo, byteorder='big')
            parametros.torneio = int.from_bytes(torneio, byteorder='big')
            parametros.fraseAlvo = fraseAlvo

    except FileNotFoundError as e:
        print(f"Arquivo {e} não foi encontrado.\n")

    return parametros

'''
    Função fitness:
        Calcula o fitness de um indivíduo baseado no número de caracteres distintos entre
        a frase alvo e a frase cópia. Quanto menor o número de caracteres distintos, melhor
        o fitness. Ambas as frases estão representadas como arrays numéricos para melhor
        compatibilidade com a biblioteca numba, usada para otimizar o código.
    Parâmetros:
        fraseArray - frase representada como array numérico do indivíduo a ser avaliado.
        fraseAlvoAray - frase alvo representada como array numérico.
    Retorno:
        Número de caracteres distintos entre a frase alvo e a frase cópia.
'''
@njit
def fitness(fraseArray, fraseAlvoAray):
    quantidadeDistintos = 0
    tamanhoCopia = len(fraseArray)    #candidato
    tamanhoAlvo = len(fraseAlvoAray)

    tamanhoMin = min(tamanhoCopia, tamanhoAlvo)   #Percorre até o menor comprimento (tamanhoMin), para não sair do array.
    for i in range(tamanhoMin):
        if fraseArray[i] != fraseAlvoAray[i]:
            quantidadeDistintos += 1    # cada caractere diferente soma 1 ponto de penalidade

    # Penalidade pela diferença de tamanho
    # Se o indivíduo for maior ou menor que a frase alvo, soma a diferença de tamanho como penalidade.
    #Isso garante que frases de tamanho incorreto nunca tenham fitness = 0, mesmo que os caracteres coincidam até certo ponto.
    quantidadeDistintos += abs(tamanhoCopia - tamanhoAlvo)

    return quantidadeDistintos

'''
    Função mutacaoAuxiliar:
        Altera aleatoriamente um gene (número) da frase do filho, substituindo-o
        por um valor aleatório correspondente a um dos caracteres presentes no alfabeto
        da frase alvo. Ambas as frases estão representadas como arrays numéricos para
        melhor compatibilidade com a biblioteca numba, usada para otimizar o código.
    Parâmetros:
        fraseArray - frase representada como array numérico do indivíduo a ser avaliado.
        taxaMutacao - taxa de mutação (1 a 100).
        tamanhoFraseAlvo - tamanho da frase alvo.
        tamanhoAlfabeto - número de caracteres disponíveis para a mutação.
    Retorno:
        Frase após ser mutada.
'''
@njit
def mutacaoAuxiliar(fraseArray, taxaMutacao, tamanhoAlfabeto):
    # Frase auxiliar
    novaFraseArray = fraseArray.copy()    # Faz uma cópia, porque não queremos alterar diretamente o indivíduo original.

    # Verifica se ocorrerá mutação baseado na taxa
    # Gera um número aleatório inteiro entre 0 e 99.
    # Se esse valor for menor ou igual à taxa de mutação, ocorre mutação.
    # Exemplo: se taxaMutacao = 15, temos 15% de chance de mutação.
    # Também checa tamanhoAlfabeto > 0 (senão não teria valor possível para substituir).

    if np.random.randint(0, 100) <= taxaMutacao and tamanhoAlfabeto > 0:
        posicao = np.random.randint(len(novaFraseArray))    # escolhe uma posição aleatória dentro do array.
        novoValor = np.random.randint(0, tamanhoAlfabeto)   # escolhe um novo caractere (representado por um número inteiro do alfabeto).
        novaFraseArray[posicao] = novoValor                 # Substitui naquela posição.

    return novaFraseArray

'''
    Função mutacao:
        Utiliza a função mutacaoAuxiliar para realizar a mutação de um indivíduo.
    Parâmetros:
        filho - um dos indivíduos da população.
        parametros - classe PARAMETROS.
    Retorno:
        Indivíduo mutado.
'''
def mutacao(filho, parametros):
    individuo = INDIVIDUO(filho.fitness, filho.fraseArray) # Cria um novo indivíduo (individuo) com os mesmos valores do filho. Isso garante que o indivíduo mutado seja independente, sem sobrescrever o original.

    # Certifica que temos a representação de array
    if filho.fraseArray is None:
        return INDIVIDUO(filho.fitness, None) # Retorna cópia sem array se original não tem

    # Aplica mutação na representação numérica. Chama a função mutacaoAuxiliar para realmente realizar a mutação.
    individuo.fraseArray = mutacaoAuxiliar(filho.fraseArray, parametros.mutacao, parametros.quantidadeLetras)

    return individuo

''' Diferença entre mutacao e mutacaoAuxiliar

mutacaoAuxiliar: função otimizada (@njit) que só mexe no array → rápida, mas trabalha em baixo nível.

mutacao: envolve a classe INDIVIDUO e faz toda a integração com o algoritmo evolutivo → cria cópia, protege contra erros e chama a auxiliar.

--> A mutacao é como o "gateway" de alto nível que o algoritmo usa para mutar indivíduos.'''

'''
    Função recombinacaoUniformeAuxiliar:
        Combina aleatoriamente os genes da frase pai e mãe no filho. Ambas as frases
        estão representadas como arrays numéricos para melhor compatibilidade com a
        biblioteca numba, usada para otimizar o código.
    Parâmetros:
        frasePaiArray - frase do pai representada como array numérico.
        fraseMaeArray - frase da mãe representada como array numérico.
        tamanho - tamanho da frase alvo.
    Retorno:
        Frase do filho resultante da combinação entre pai e mãe (representada como array numérico).
'''
@njit
def recombinacaoUniformeAuxiliar(frasePaiArray, fraseMaeArray, tamanho):
    '''O filho não pode ser maior que:
      o pai,
      a mãe,
      e o tamanho definido como limite.'''
    comprimentoMax = min(len(frasePaiArray), len(fraseMaeArray), tamanho)
    filhoArray = np.empty(comprimentoMax, dtype=frasePaiArray.dtype)    #Cria um array vazio (np.empty) para armazenar os genes do filho.

    '''Para cada posição do filho:

        Sorteia aleatoriamente (0 ou 1).

        Se deu 1, pega o gene do pai.

        Se deu 0, pega o gene da mãe.

        --> Isso é crossover uniforme: cada posição é herdada aleatoriamente de um dos pais, com probabilidade igual.'''

    for i in range(comprimentoMax):
        if np.random.randint(0, 2) == 1:
            filhoArray[i] = frasePaiArray[i]
        else:
            filhoArray[i] = fraseMaeArray[i]

    return filhoArray

'''
    Função recombinacaoUniforme:
        Utiliza a função recombinacaoUniformeAuxiliar para realizar a recombinação entre pai e mãe.
    Parâmetros:
        pai - um dos indivíduos da população.
        mae - um dos indivíduos da população.
        numero - tamanho da frase alvo.
    Retorno:
        Indivíduo filho resultante da combinação entre pai e mãe.
'''
def recombinacaoUniforme(pai, mae, numero, parametros):
    filho = INDIVIDUO()

    # Certifica que os pais têm representação de array
    if pai.fraseArray is None or mae.fraseArray is None:
        return INDIVIDUO() # Retorna indivíduo vazio

    filho.fraseArray = recombinacaoUniformeAuxiliar(pai.fraseArray, mae.fraseArray, numero) # Chama a função auxiliar que faz o crossover gene a gene (cada posição é escolhida aleatoriamente entre pai e mãe).

    return filho

'''
    Função selecaoPorTorneio:
        Seleciona dois a dois indivíduos da população e determina qual deles possui o menor fitness.
    Parâmetros:
        populacao - população de indivíduos (frases cópias).
        parametros - classe PARAMETROS.
    Retorno:
        Retorna o indivíduo com o menor fitness entre os selecionados.
'''
# Essa função não altera os indivíduos, apenas seleciona o “melhor” para reprodução.
def selecaoPorTorneio(populacao, parametros):
    melhor = INDIVIDUO()    # indivíduo fictício para servir de referência inicial.
    melhor.fitness = -1

    for i in range(parametros.torneio):   # itera parametros.torneio vezes
        auxiliar = populacao[np.random.randint(0, parametros.populacao)]    # Sorteia aleatoriamente um indivíduo da população.
        if melhor.fitness == -1 or auxiliar.fitness < melhor.fitness:   # Compara o fitness do sorteado com o melhor atual.
            melhor = auxiliar

    return melhor

'''
    Função elitismo:
        Calcula o número de indivíduos da população que não sofreram ação dos operadores de mutação
        e recombinação na função reprodução e ordena por adaptação (indivíduos mais adaptados
        ocupam posições iniciais).
    Parâmetros:
        população - população de indivíduos (frases cópias).
        parametros - classe PARAMETROS.
    Retorno:
        Número de indivíduos selecionados.
'''
# Calcula quantos indivíduos serão mantidos
def elitismo(populacao, parametros):
    '''parametros.elitismo é a taxa de elitismo em % (ex: 5%).

      Multiplicando pelo tamanho da população, obtemos quantos indivíduos mais adaptados serão mantidos sem alterações.'''
    selecionados = parametros.populacao * parametros.elitismo // 100

    '''Ordena os indivíduos do melhor (menor fitness) para o pior (maior fitness).

      Assim, os primeiros selecionados da lista são os que serão preservados.'''
    populacao.sort(key=lambda individuo: individuo.fitness)

    return selecionados

'''
    Função reproducao:
        Aplica as funções de elitismo, torneio, recombinação e mutação na população.
    Parâmetros:
        populacao - população de indivíduos (frases cópias).
        parametros - classe PARAMETROS.
    Retorno:
        Retorna o melhor indivíduo da população.
'''
def reproducao(populacao, parametros):
    melhor = INDIVIDUO()
    pai = INDIVIDUO()
    mae = INDIVIDUO()
    filho = INDIVIDUO()

    '''array temporário para armazenar os indivíduos da próxima geração., vai gerar uma lista de tamanho parametros.populacao.

    Cada elemento da lista é uma nova instância da classe INDIVIDUO, criada chamando INDIVIDUO()'''
    novaPopulacao = [INDIVIDUO() for _ in range(parametros.populacao)]

    taxaDeElitismo = elitismo(populacao, parametros)

    melhor = populacao[0]

    '''elitismo retorna quantos indivíduos serão preservados.

        populacao.sort dentro de elitismo já deixou os melhores no início da lista.

        Copia os indivíduos mais adaptados para novaPopulacao sem alterações.'''

    for i in range(taxaDeElitismo):
        novaPopulacao[i] = INDIVIDUO(populacao[i].fitness, np.copy(populacao[i].fraseArray))

    '''Cria filhos para o resto da população

      Para cada posição que não está protegida pelo elitismo:

        Seleciona pai e mãe via torneio.

        Faz crossover (recombinacaoUniforme) → filho herda genes dos pais.

        Aplica mutação → pequenas mudanças aleatórias nos genes do filho.

        Calcula fitness do filho → quão próximo ele está da frase alvo.

        Atualiza melhor se o filho for mais adaptado.

        Adiciona filho em novaPopulacao.
    '''

    for i in range(taxaDeElitismo, parametros.populacao):
        pai = selecaoPorTorneio(populacao, parametros)
        mae = selecaoPorTorneio(populacao, parametros)
        filho = recombinacaoUniforme(pai, mae, len(parametros.fraseAlvoArray), parametros)
        filho = mutacao(filho, parametros)
        filho.fitness = fitness(filho.fraseArray, parametros.fraseAlvoArray)

        if filho.fitness < melhor.fitness:
            melhor = filho
        novaPopulacao[i] = filho


    # Atualiza a população original
    for i in range(taxaDeElitismo, parametros.populacao):
        populacao[i] = novaPopulacao[i]


    # A função retorna o indivíduo mais adaptado da geração, que será usado no loop principal para monitoramento.
    return melhor

'''
    Função geraFrasesAleatorias:
        Gera frases aleatórias de tamanho definido, utilizando os caracteres disponíveis
        no alfabeto. As frases geradas são representadas como arrays numéricos para
        melhor compatibilidade com a biblioteca numba, usada para otimizar o código.
    Parâmetros:
        tamanhoPopulacao - número de frases a serem geradas.
        tamanhoFrase - tamanho de cada frase (representada em array numérico).
        numLetras - número de caracteres distintos na frase alvo.
    Retorno:
        Lista de frases aleatórias geradas.
'''

# Essa função cria a população inicial, com frases aleatórias codificadas numericamente.
@njit
def geraFrasesAleatorias(tamanhoPopulacao, tamanhoFrase, numLetras):
    frases = [np.empty(tamanhoFrase, dtype=np.int64) for _ in range(tamanhoPopulacao)]
    for i in range(tamanhoPopulacao):
        frases[i] = np.random.randint(0, numLetras, size=tamanhoFrase)

    return frases

'''
    Função inicializa:
        Utiliza a função geraFrasesAleatorias para inicializar a população com frases
        aleatórias (representadas como arrays numéricos) de tamanho definido e
        calcula o fitness de cada uma delas.
    Parâmetros:
        populacao - população de indivíduos (frases cópias).
        parametros - classe PARAMETROS.
    Retorno:
        Nulo.
'''

def inicializa(populacao, parametros):
    numLetras = parametros.quantidadeLetras
    tamanhoFrase = len(parametros.fraseAlvo)
    tamanhoPopulacao = parametros.populacao

    frasesGeradas = geraFrasesAleatorias(tamanhoPopulacao, tamanhoFrase, numLetras)


    '''Para cada posição na população:

      Cria um novo indivíduo (INDIVIDUO()).

      Atribui a frase aleatória gerada (fraseArray).

      Calcula o fitness do indivíduo:

      fitness(fraseArray, fraseAlvoArray) → número de diferenças entre a frase do indivíduo e a frase alvo.

      Quanto menor, melhor.'''
    for i in range(tamanhoPopulacao):
        populacao[i] = INDIVIDUO()
        populacao[i].fraseArray = frasesGeradas[i]
        populacao[i].fitness = fitness(populacao[i].fraseArray, parametros.fraseAlvoArray)

        '''Ao final, populacao está totalmente inicializada:

            Cada indivíduo tem uma frase aleatória.

            Cada indivíduo tem seu fitness calculado.

            Essa população é a linha de partida para a evolução.'''

'''
    Função alfabeto:
        Determina quais caracteres compõem a frase alvo.
    Parâmetros:
        parametros - classe PARAMETROS.
    Retorno:
        Retorna a classe PARAMETROS com a quantidade de letras, mapeamento
        entre char <-> int e frase alvo convertida em array numérico.
'''
def alfabeto(parametros):
    alfabeto = "".join(sorted(set(parametros.fraseAlvo)))

    parametros.quantidadeLetras = len(alfabeto)

    # Cria mapeamentos char <-> int
    parametros.charToInt = {char: i for i, char in enumerate(alfabeto)}
    parametros.intToChar = np.array(list(alfabeto), dtype=str) # Array para mapeamento reverso rápido

    # Converter fraseAlvo para array numérico (mantendo a string original)
    parametros.fraseAlvoArray = np.array([parametros.charToInt[c] for c in parametros.fraseAlvo], dtype=np.int32)


    return parametros

'''
    Função setSeed:
        Define uma semente para o gerador de números aleatórios usado pela biblioteca
        numba, permitindo reprodutibilidade nos testes.
    Parâmetros:
        seed - semente a ser definida.
    Retorno:
        Nulo.
'''
'''@njit
def setSeed(seed):
    np.random.seed(seed)'''

'''
    Função main:
        Implementa o Algoritmo Evolutivo para o problema: dada uma frase alvo,
        reproduza-a através de uma população de frases cópias geradas aleatoriamente.
'''

def main():
    #np.random.seed(0) # Semente fixa para testes
    #setSeed(0) # Semente fixa para testes


    inicio = time.time()

    escreveArquivo()    # cria o arquivo entrada.in com os parâmetros do teste e a frase alvo.
    parametros = leArquivo()    # lê o arquivo entrada.in e retorna os parâmetros como uma instância da classe PARAMETROS.
    parametros = alfabeto(parametros)   # cria os mapeamentos char ↔ int e transforma a frase alvo em um array numérico.

    #print(f"{len(parametros.fraseAlvo)}\n")

    geracao = 0
    populacao = [INDIVIDUO() for _ in range(parametros.populacao)]      # Cria uma lista de objetos INDIVIDUO, com tamanho igual à população definida.

    np.random.seed(int(time.time())) # Semente de acordo com o tempo de máquina
    inicializa(populacao, parametros)   # Chama inicializa para gerar frases aleatórias e calcular fitness.


    '''O loop é o ciclo principal do algoritmo evolutivo: gerar população → avaliar fitness → reproduzir → mutação → selecionar o melhor.'''
    while geracao <= parametros.geracoes:
        melhor = reproducao(populacao, parametros)    # aplica elitismo, seleção por torneio, crossover e mutação e retorna o melhor indivíduo da geração.
        print(f"\nIteracao {geracao}, melhor fitness {melhor.fitness}.\n")
        # Converte a frase do melhor indivíduo para string
        melhorFrase = "".join(parametros.intToChar[melhor.fraseArray]) if melhor.fraseArray is not None else "N/A"
        print(f"{melhorFrase}\n")
        geracao += 1
        if melhor.fitness == 0:
            break

    fim = time.time()
    total = fim - inicio
    print(f"Tempo total gasto pela CPU: {total}")
    escreveRelatorio(total, melhor.fitness)

if __name__ == "__main__":
    main()


A saída de streaming foi truncada nas últimas 5000 linhas.
Iteracao 801, melhor fitness 24.

O Tejo e maiscbelo que o rio que correfpela minja amdeia,Mas o Tejo nao e mais belo que o rio que corre pela minha aldeia.Porque o Tejo nao e o rio que corre pela minha aldeia,O Tejo tem grandes navios,E nele navega ainda,P,ra aqueles que veem emrtudo o q.e la nao esta,A memoria das naus.O Tejo desce de Espanha,E o Tejo entra noumar em Portugal.Toda a gente sabe isso.Mas poucoq sabem qual   ovrib da mizha aldeia,E para onde ele vai,E donde ele vem.E por isso, porque pertence a menos gente,e mais livre e mazor o rio da minha aldeiaiPelo Tejo vai se para o Mundo.Para aleN do Teju ha a America.E a fortuna daqueles que a encontram.Ninguem nunca censou no que ha para alem.Do rio da minha aldeia.O rio da minha aldeia nao faz pen,ar em nada. uen esta ae pe dele esta so ab pe dede.


Iteracao 802, melhor fitness 24.

O Tejo e maiscbelo que o rio que correfpela minja amdeia,Mas o Tejo nao e mais belo qu

In [34]:
import numpy as np
import random

# Representação do indivíduo
class Individuo:
    def __init__(self, movimentos):
        self.movimentos = movimentos  # lista de direções
        self.x = None
        self.y = None
        self.energia = None

# Gera caminhos aleatórios (sequência de movimentos)
def gerar_individuo(tam):
    movimentos = [random.randint(0, 3) for _ in range(tam-1)]  # tam-1 movimentos
    return Individuo(movimentos)

# Constrói coordenadas a partir dos movimentos
def construir_coordenadas(ind):
    x, y = [0], [0]
    for mov in ind.movimentos:
        if mov == 0:  # direita
            x.append(x[-1] + 1)
            y.append(y[-1])
        elif mov == 1:  # cima
            x.append(x[-1])
            y.append(y[-1] + 1)
        elif mov == 2:  # esquerda
            x.append(x[-1] - 1)
            y.append(y[-1])
        elif mov == 3:  # baixo
            x.append(x[-1])
            y.append(y[-1] - 1)
    ind.x, ind.y = x, y

# Calcula energia simplificada (H-H adjacentes não consecutivos = -1)
def calcular_energia(ind, cadeia):
    construir_coordenadas(ind)
    energia = 0
    n = len(cadeia)
    for i in range(n):
        if cadeia[i] != 'H':
            continue
        for j in range(i+2, n):  # ignora vizinhos consecutivos
            if cadeia[j] != 'H':
                continue
            if abs(ind.x[i] - ind.x[j]) + abs(ind.y[i] - ind.y[j]) == 1:
                energia -= 1
    ind.energia = energia

# Seleção por torneio
def selecao_torneio(populacao, k=3):
    escolhidos = random.sample(populacao, k)
    return min(escolhidos, key=lambda ind: ind.energia)

# Crossover de 1 ponto
def crossover(pai, mae):
    ponto = random.randint(1, len(pai.movimentos)-1)
    filho_mov = pai.movimentos[:ponto] + mae.movimentos[ponto:]
    return Individuo(filho_mov)

# Mutação: altera movimento aleatório
def mutacao(ind, taxa=0.1):
    filho = Individuo(ind.movimentos[:])
    if random.random() < taxa:
        pos = random.randint(0, len(filho.movimentos)-1)
        filho.movimentos[pos] = random.randint(0, 3)
    return filho

# Loop evolutivo simplificado
def algoritmo_evolutivo(cadeia, pop_size=100, geracoes=200, taxa_mut=0.2, elitismo=5):
    tam = len(cadeia)
    populacao = [gerar_individuo(tam) for _ in range(pop_size)]
    for ind in populacao:
        calcular_energia(ind, cadeia)

    melhor = min(populacao, key=lambda ind: ind.energia)

    for g in range(geracoes):
        nova_pop = []
        populacao.sort(key=lambda ind: ind.energia)
        nova_pop.extend(populacao[:elitismo])  # elitismo

        while len(nova_pop) < pop_size:
            pai = selecao_torneio(populacao)
            mae = selecao_torneio(populacao)
            filho = crossover(pai, mae)
            filho = mutacao(filho, taxa_mut)
            calcular_energia(filho, cadeia)
            nova_pop.append(filho)

        populacao = nova_pop
        atual = min(populacao, key=lambda ind: ind.energia)
        if atual.energia < melhor.energia:
            melhor = atual

        print(f"Geração {g}: melhor energia = {melhor.energia}")

    return melhor

# -------------------------------
# Execução de teste rápido
# -------------------------------
if __name__ == "__main__":
    # Lê sequência do arquivo
    with open("protein24.txt") as f:
        n = int(f.readline().strip())
        cadeia = f.readline().strip()

    melhor = algoritmo_evolutivo(cadeia, pop_size=50, geracoes=100)
    print("\nMelhor solução encontrada:")
    print("Movimentos:", melhor.movimentos)
    print("Energia:", melhor.energia)


Geração 0: melhor energia = -19
Geração 1: melhor energia = -19
Geração 2: melhor energia = -19
Geração 3: melhor energia = -19
Geração 4: melhor energia = -19
Geração 5: melhor energia = -19
Geração 6: melhor energia = -19
Geração 7: melhor energia = -20
Geração 8: melhor energia = -20
Geração 9: melhor energia = -20
Geração 10: melhor energia = -20
Geração 11: melhor energia = -20
Geração 12: melhor energia = -20
Geração 13: melhor energia = -20
Geração 14: melhor energia = -20
Geração 15: melhor energia = -20
Geração 16: melhor energia = -20
Geração 17: melhor energia = -20
Geração 18: melhor energia = -20
Geração 19: melhor energia = -20
Geração 20: melhor energia = -21
Geração 21: melhor energia = -21
Geração 22: melhor energia = -21
Geração 23: melhor energia = -21
Geração 24: melhor energia = -21
Geração 25: melhor energia = -21
Geração 26: melhor energia = -21
Geração 27: melhor energia = -21
Geração 28: melhor energia = -21
Geração 29: melhor energia = -21
Geração 30: melhor e

In [36]:
import numpy as np
import random
import math

class Individuo:
    def __init__(self, movimentos):
        self.movimentos = movimentos
        self.x = None
        self.y = None
        self.energia = None

def gerar_individuo(tam):
    x, y = 0, 0
    visitados = {(0,0)}
    movimentos = []

    for _ in range(tam-1):
        possiveis = []
        candidatos = [(1,0,0), (0,1,1), (-1,0,2), (0,-1,3)]  # (dx,dy,mov)

        for dx,dy,mov in candidatos:
            nx, ny = x+dx, y+dy
            if (nx,ny) not in visitados:
                possiveis.append((nx,ny,mov))

        if not possiveis:
            # sem saída -> retorna indivíduo inválido (energia inf)
            return Individuo([random.randint(0,3) for _ in range(tam-1)])

        nx, ny, mov = random.choice(possiveis)
        movimentos.append(mov)
        x, y = nx, ny
        visitados.add((x,y))

    return Individuo(movimentos)


def construir_coordenadas(ind):
    x, y = [0], [0]
    visitados = {(0,0)}  # guarda posições já ocupadas
    valido = True

    for mov in ind.movimentos:
        if mov == 0:   # direita
            nx, ny = x[-1] + 1, y[-1]
        elif mov == 1: # cima
            nx, ny = x[-1], y[-1] + 1
        elif mov == 2: # esquerda
            nx, ny = x[-1] - 1, y[-1]
        else:          # baixo
            nx, ny = x[-1], y[-1] - 1

        if (nx, ny) in visitados:  # colisão -> dobra inválida
            valido = False
        visitados.add((nx, ny))
        x.append(nx)
        y.append(ny)

    ind.x, ind.y = x, y
    return valido

def calcular_energia(ind, cadeia):
    valido = construir_coordenadas(ind)
    if not valido:
        ind.energia = math.inf  # penaliza fortemente colisões
        return

    energia = 0
    n = len(cadeia)
    for i in range(n):
        if cadeia[i] != 'H':
            continue
        for j in range(i+2, n):  # ignora vizinhos consecutivos
            if cadeia[j] != 'H':
                continue
            if abs(ind.x[i] - ind.x[j]) + abs(ind.y[i] - ind.y[j]) == 1:
                energia -= 1
    ind.energia = energia

def selecao_torneio(populacao, k=3):
    escolhidos = random.sample(populacao, k)
    return min(escolhidos, key=lambda ind: ind.energia)

def crossover(pai, mae):
    ponto = random.randint(1, len(pai.movimentos)-1)
    filho_mov = pai.movimentos[:ponto] + mae.movimentos[ponto:]
    return Individuo(filho_mov)

def mutacao(ind, taxa=0.1):
    filho = Individuo(ind.movimentos[:])
    if random.random() < taxa:
        pos = random.randint(0, len(filho.movimentos)-1)
        filho.movimentos[pos] = random.randint(0, 3)
    return filho

def algoritmo_evolutivo(cadeia, pop_size=100, geracoes=200, taxa_mut=0.2, elitismo=5):
    tam = len(cadeia)
    populacao = [gerar_individuo(tam) for _ in range(pop_size)]
    for ind in populacao:
        calcular_energia(ind, cadeia)

    melhor = min(populacao, key=lambda ind: ind.energia)

    for g in range(geracoes):
        nova_pop = []
        populacao.sort(key=lambda ind: ind.energia)
        nova_pop.extend(populacao[:elitismo])  # elitismo

        while len(nova_pop) < pop_size:
            pai = selecao_torneio(populacao)
            mae = selecao_torneio(populacao)
            filho = crossover(pai, mae)
            filho = mutacao(filho, taxa_mut)
            calcular_energia(filho, cadeia)
            nova_pop.append(filho)

        populacao = nova_pop
        atual = min(populacao, key=lambda ind: ind.energia)
        if atual.energia < melhor.energia:
            melhor = atual

        print(f"Geração {g}: melhor energia = {melhor.energia}")

    return melhor

if __name__ == "__main__":
    with open("protein24.txt") as f:
        n = int(f.readline().strip())
        cadeia = f.readline().strip()

    melhor = algoritmo_evolutivo(cadeia, pop_size=50, geracoes=100)
    print("\nMelhor solução encontrada:")
    print("Movimentos:", melhor.movimentos)
    print("Energia:", melhor.energia)


Geração 0: melhor energia = -4
Geração 1: melhor energia = -4
Geração 2: melhor energia = -4
Geração 3: melhor energia = -5
Geração 4: melhor energia = -5
Geração 5: melhor energia = -5
Geração 6: melhor energia = -5
Geração 7: melhor energia = -5
Geração 8: melhor energia = -5
Geração 9: melhor energia = -5
Geração 10: melhor energia = -5
Geração 11: melhor energia = -5
Geração 12: melhor energia = -5
Geração 13: melhor energia = -5
Geração 14: melhor energia = -5
Geração 15: melhor energia = -5
Geração 16: melhor energia = -5
Geração 17: melhor energia = -5
Geração 18: melhor energia = -5
Geração 19: melhor energia = -5
Geração 20: melhor energia = -5
Geração 21: melhor energia = -5
Geração 22: melhor energia = -5
Geração 23: melhor energia = -5
Geração 24: melhor energia = -5
Geração 25: melhor energia = -5
Geração 26: melhor energia = -5
Geração 27: melhor energia = -5
Geração 28: melhor energia = -5
Geração 29: melhor energia = -5
Geração 30: melhor energia = -5
Geração 31: melhor